In [16]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
c:\Users\jaesc2\GitHub\skforecast


In [17]:
import re
import pytest
import joblib
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.exceptions import NotFittedError
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import HistGradientBoostingRegressor
from lightgbm import LGBMRegressor

from skforecast.exceptions import IgnoredArgumentWarning
from skforecast.preprocessing import RollingFeatures
from skforecast.preprocessing import TimeSeriesDifferentiator
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregDirect import ForecasterAutoregDirect
from skforecast.ForecasterAutoregMultiSeries import ForecasterAutoregMultiSeries

from skforecast.ForecasterAutoreg.tests.fixtures_ForecasterAutoreg import data  # to test results when using differentiation


In [18]:
from skforecast.ForecasterAutoregMultiSeries.tests.fixtures_ForecasterAutoregMultiSeries import series
from skforecast.ForecasterAutoregMultiSeries.tests.fixtures_ForecasterAutoregMultiSeries import exog
from skforecast.ForecasterAutoregMultiSeries.tests.fixtures_ForecasterAutoregMultiSeries import exog_predict

series_dict = joblib.load(r'C:\Users\jaesc2\GitHub\skforecast\skforecast\ForecasterAutoregMultiSeries\tests\fixture_sample_multi_series.joblib')
exog_dict = joblib.load(r'C:\Users\jaesc2\GitHub\skforecast\skforecast\ForecasterAutoregMultiSeries\tests\fixture_sample_multi_series_exog.joblib')
end_train = "2016-07-31 23:59:00"
series_dict_train = {k: v.loc[:end_train,] for k, v in series_dict.items()}
exog_dict_train = {k: v.loc[:end_train,] for k, v in exog_dict.items()}
series_dict_test = {k: v.loc[end_train:,] for k, v in series_dict.items()}
exog_dict_test = {k: v.loc[end_train:,] for k, v in exog_dict.items()}

series_2 = pd.DataFrame({'1': pd.Series(np.arange(start=0, stop=50, dtype=float)), 
                         '2': pd.Series(np.arange(start=50, stop=100, dtype=float))})

In [24]:
series = {
    'l1': pd.Series(np.arange(10, dtype=float)), 
    'l2': pd.Series(np.arange(15, 20, dtype=float)),
    'l3': pd.Series(np.arange(20, 25, dtype=float))
}
series['l1'].loc[3] = np.nan
series['l2'].loc[2] = np.nan
series['l1'].index = pd.date_range("1990-01-01", periods=10, freq='D')
series['l2'].index = pd.date_range("1990-01-05", periods=5, freq='D')
series['l3'].index = pd.date_range("1990-01-03", periods=5, freq='D')

exog = {
    'l1': pd.Series(np.arange(100, 110), name='exog_1', dtype=float),
    'l2': None,
    'l3': pd.DataFrame({'exog_1': np.arange(203, 209, dtype=float),
                        'exog_2': ['a', 'b', 'a', 'b', 'a', 'b']})
}
exog['l1'].index = pd.date_range("1990-01-01", periods=10, freq='D')
exog['l3'].index = pd.date_range("1990-01-03", periods=6, freq='D')

rolling = RollingFeatures(stats=['mean', 'median'], window_sizes=[3, 3])
rolling_2 = RollingFeatures(stats='sum', window_sizes=4)

forecaster = ForecasterAutoregMultiSeries(
    LinearRegression(), lags=3,
    encoding='onehot',
    window_features=[rolling, rolling_2],
    transformer_series=None,
    dropna_from_series=False
)
results = forecaster._create_train_X_y(series=series, exog=exog)
results

c:\Users\jaesc2\GitHub\skforecast\skforecast\ForecasterAutoregMultiSeries\ForecasterAutoregMultiSeries.py:431: UserWarning: When using a linear model, it is recommended to use a transformer_series to ensure all series are in the same scale. You can use, for example, a `StandardScaler` from sklearn.preprocessing.
  warnings.warn(
c:\Users\jaesc2\GitHub\skforecast\skforecast\utils\utils.py:553: DataTypeWarning: `exog` may contain only `int`, `float` or `category` dtypes. Most machine learning models do not allow other types of values. Fitting the forecaster may fail. 
 You can suppress this warning using: warnings.simplefilter('ignore', category=ValueTypesExogWarning)
  warnings.warn(
c:\Users\jaesc2\GitHub\skforecast\skforecast\ForecasterAutoregMultiSeries\ForecasterAutoregMultiSeries.py:1014: MissingValuesWarning: NaNs detected in `X_train`. Some regressors do not allow NaN values during training. If you want to drop them, set `forecaster.dropna_from_series = True`. 
 You can suppress 

(            lag_1  lag_2  lag_3  roll_mean_3  roll_median_3  roll_sum_4  l1  \
 1990-01-05    NaN    2.0    1.0          NaN            NaN         NaN   1   
 1990-01-06    4.0    NaN    2.0          NaN            NaN         NaN   1   
 1990-01-07    5.0    4.0    NaN          NaN            NaN         NaN   1   
 1990-01-08    6.0    5.0    4.0          5.0            5.0         NaN   1   
 1990-01-09    7.0    6.0    5.0          6.0            6.0        22.0   1   
 1990-01-10    8.0    7.0    6.0          7.0            7.0        26.0   1   
 1990-01-09   18.0    NaN   16.0          NaN            NaN         NaN   0   
 1990-01-07   23.0   22.0   21.0         22.0           22.0        86.0   0   
 
             l2  l3  exog_1 exog_2  
 1990-01-05   0   0   104.0    NaN  
 1990-01-06   0   0   105.0    NaN  
 1990-01-07   0   0   106.0    NaN  
 1990-01-08   0   0   107.0    NaN  
 1990-01-09   0   0   108.0    NaN  
 1990-01-10   0   0   109.0    NaN  
 1990-01-09   1   0

In [25]:
results[0]

,lag_1,lag_2,lag_3,roll_mean_3,roll_median_3,roll_sum_4,l1,l2,l3,exog_1,exog_2
1990-01-05,NaN,2.0,1.0,NaN,NaN,NaN,1,0,0,104.0,NaN
1990-01-06,4.0,NaN,2.0,NaN,NaN,NaN,1,0,0,105.0,NaN
1990-01-07,5.0,4.0,NaN,NaN,NaN,NaN,1,0,0,106.0,NaN
1990-01-08,6.0,5.0,4.0,5.0,5.0,NaN,1,0,0,107.0,NaN
1990-01-09,7.0,6.0,5.0,6.0,6.0,22.0,1,0,0,108.0,NaN
1990-01-10,8.0,7.0,6.0,7.0,7.0,26.0,1,0,0,109.0,NaN
1990-01-09,18.0,NaN,16.0,NaN,NaN,NaN,0,1,0,NaN,NaN
1990-01-07,23.0,22.0,21.0,22.0,22.0,86.0,0,0,1,207.0,a


In [27]:
results[1].to_numpy()

array([ 4.,  5.,  6.,  7.,  8.,  9., 19., 24.])

In [4]:
end_train = '2003-03-01 23:59:00'

# Simulated exogenous variable
rng = np.random.default_rng(9876)
exog = pd.Series(
    rng.normal(loc=0, scale=1, size=len(data)), index=data.index, name='exog'
)
rolling = RollingFeatures(stats=['mean', 'median'], window_sizes=[5, 5])
rolling_2 = RollingFeatures(stats='sum', window_sizes=[6])

forecaster = ForecasterAutoregDirect(
                    regressor        = LinearRegression(),
                    lags             = [1, 5],
                    window_features  = [rolling, rolling_2],
                    steps            = 6,
                    transformer_y    = None,
                    transformer_exog = None,
                    differentiation  = None
                )
forecaster.fit(y=data.loc[:end_train], exog=exog.loc[:end_train])
X_predict = forecaster.create_predict_X(exog=exog.loc[end_train:])
X_predict

,lag_1,lag_5,roll_mean_5,roll_median_5,roll_sum_6,exog
2003-04-01,2.043333,2.366796,2.261869,2.366796,13.686322,1.161729
2003-05-01,2.043333,2.366796,2.261869,2.366796,13.686322,0.294688
2003-06-01,2.043333,2.366796,2.261869,2.366796,13.686322,-0.439976
2003-07-01,2.043333,2.366796,2.261869,2.366796,13.686322,1.250084
2003-08-01,2.043333,2.366796,2.261869,2.366796,13.686322,1.374969
2003-09-01,2.043333,2.366796,2.261869,2.366796,13.686322,-0.416732


In [8]:
for i, step in enumerate(range(1, forecaster.steps + 1)):
    print(forecaster.regressors_[step].predict(X_predict.iloc[[i]]))

[2.19238587]
[2.23555505]
[2.25654547]
[2.34763646]
[2.31040474]
[2.43900132]


In [10]:
forecaster.predict(steps=[2], exog=exog.loc[end_train:]).to_numpy()

array([2.23555505])

In [11]:
import pytest
import re
import numpy as np
import pandas as pd
from skforecast.model_selection._split import BaseFold

In [12]:
valid_params = {
    "steps": 5,
    "initial_train_size": 100,
    "window_size": 10,
    "differentiation": None,
    "refit": True,
    "fixed_train_size": True,
    "gap": 0,
    "skip_folds": None,
    "allow_incomplete_fold": True,
    "return_all_indexes": False,
    "verbose": True,
}

In [13]:
cv = BaseFold()
params = dict(valid_params)
params["initial_train_size"] = "invalid"
msg = (
    f"`initial_train_size` must be an integer greater than 0 or None. "
    f"Got {params["initial_train_size"]}."
)
with pytest.raises(ValueError, match=msg):
    cv._validate_params(cv_name="TimeSeriesFold", **params)

In [15]:
params is valid_params

False

In [14]:
params

{'steps': 5,
 'initial_train_size': 'invalid',
 'window_size': 10,
 'differentiation': None,
 'refit': True,
 'fixed_train_size': True,
 'gap': 0,
 'skip_folds': None,
 'allow_incomplete_fold': True,
 'return_all_indexes': False,
 'verbose': True}

In [28]:
pd.DataFrame(
    data = np.array([[np.nan, 2., 1., np.nan, np.nan, np.nan, 1., 0., 0., 104., np.nan],
                        [4., np.nan, 2., np.nan, np.nan, np.nan, 1., 0., 0., 105., np.nan],
                        [5., 4., np.nan, np.nan, np.nan, np.nan, 1., 0., 0., 106., np.nan],
                        [6., 5., 4., 5.0, 5.0, np.nan, 1., 0., 0., 107., np.nan],
                        [7., 6., 5., 6.0, 6.0, 22.0, 1., 0., 0., 108., np.nan],
                        [8., 7., 6., 7.0, 7.0, 26.0, 1., 0., 0., 109., np.nan],
                        [18., np.nan, 16., np.nan, np.nan, np.nan, 0., 1., 0., np.nan, np.nan],
                        [23., 22., 21., 22.0, 22.0, 86.0, 0., 0., 1., 207., 'a']]),
    index   = pd.Index(
                    pd.DatetimeIndex(
                        ['1990-01-05', '1990-01-06', '1990-01-07', '1990-01-08',
                        '1990-01-09', '1990-01-10',
                        '1990-01-09', '1990-01-07']
                    )
                ),
    columns = ['lag_1', 'lag_2', 'lag_3', 'roll_mean_3', 'roll_median_3', 'roll_sum_4',
                'l1', 'l2', 'l3', 'exog_1', 'exog_2']
).astype({'lag_1': float, 'lag_2': float, 'lag_3': float, 
            'roll_mean_3': float, 'roll_median_3': float, 'roll_sum_4': float, 
            'l1': float, 'l2': float, 'l3': float, 'exog_1': float, 'exog_2': object}
).astype({'l1': int, 'l2': int, 'l3': int}
)

,lag_1,lag_2,lag_3,roll_mean_3,roll_median_3,roll_sum_4,l1,l2,l3,exog_1,exog_2
1990-01-05,NaN,2.0,1.0,NaN,NaN,NaN,1,0,0,104.0,nan
1990-01-06,4.0,NaN,2.0,NaN,NaN,NaN,1,0,0,105.0,nan
1990-01-07,5.0,4.0,NaN,NaN,NaN,NaN,1,0,0,106.0,nan
1990-01-08,6.0,5.0,4.0,5.0,5.0,NaN,1,0,0,107.0,nan
1990-01-09,7.0,6.0,5.0,6.0,6.0,22.0,1,0,0,108.0,nan
1990-01-10,8.0,7.0,6.0,7.0,7.0,26.0,1,0,0,109.0,nan
1990-01-09,18.0,NaN,16.0,NaN,NaN,NaN,0,1,0,NaN,nan
1990-01-07,23.0,22.0,21.0,22.0,22.0,86.0,0,0,1,207.0,a
